# Restricted versus unrestricted

## High spin states

### Single-reference methods

By a high-spin state, we refer to an open-shell system where all unpaired electrons are chosen to have $\alpha$-spin. As detailed in  section {ref}`sec:spin`, this case is particularly simple since the system can be described by a single Slater determinant.

There are two main ways to proceed:

* Keep the molecular orbitals of the $\alpha$ and $\beta$ spin-orbitals identical but populating the $\alpha$ orbitals with more electrons. This is known as a restricted open-shell calculation.

* Allow for different molecular orbitals for the $\alpha$ and $\beta$ spin-orbitals. This is known as an unrestricted calculation.

Let us illustrate these two approaches on the most important open-shell molecule on earth namely the oxygen molecule.

In [1]:
import multipsi as mtp
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.


In [2]:
au2kcal = 627.51

In [3]:
mol_str = """
O   0.0  0.0  -0.6
O   0.0  0.0   0.6
"""
molecule = vlx.Molecule.read_str(mol_str, units="angstrom")
basis = vlx.MolecularBasis.read(molecule, "cc-pvdz")

* Info * Reading basis set from file: /opt/miniconda3/envs/echem/lib/python3.9/site-packages/veloxchem/basis/CC-PVDZ      
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                                  Basis: CC-PVDZ                                                                          
                                                                                                                          
                                  Atom Contracted GTOs          Primitive GTOs                                            
                                                                                                                          
                

We determine the triplet ground state with the RODFT and UDFT approaches.

In [4]:
molecule.set_multiplicity(3)

triplet_rodft_drv = vlx.ScfRestrictedOpenDriver()
triplet_rodft_drv.xcfun = "B3LYP"

triplet_rodft_results = triplet_rodft_drv.compute(molecule, basis)

triplet_udft_drv = vlx.ScfUnrestrictedDriver()
triplet_udft_drv.xcfun = "B3LYP"

triplet_udft_results = triplet_udft_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Open-Shell Kohn-Sham                                 
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

Owing to the larger flexibility in the reference state, the UDFT energy is slightly lower than the RODFT one.

In [5]:
print(f"ROB3LYP energy: {triplet_rodft_drv.get_scf_energy() : 14.8f}")
print(f" UB3LYP energy: {triplet_udft_drv.get_scf_energy() : 14.8f}")

ROB3LYP energy:  -150.33015042
 UB3LYP energy:  -150.33393255


In the RODFT case, we have one set of MOs with the first seven being doubly occupied and MOs eight and nine being singly occupied.

In [6]:
triplet_rodft_drv.molecular_orbitals.print_orbitals(molecule, basis)

                                                                                                                          
                                                 Spin Restricted Orbitals                                                 
                                                 ------------------------                                                 
                                                                                                                          
               Molecular Orbital No.   4:                                                                                 
               --------------------------                                                                                 
               Occupation: 2.000 Energy:   -0.78650 a.u.                                                                  
               (   1 O   2s  :     0.36) (   1 O   3s  :     0.46) (   2 O   2s  :    -0.36)                              
               (

In the UDFT case, there are two different sets of MOs with nine occupied $\alpha$-spin orbitals and seven occupied $\beta$-spin orbitals.

In [7]:
triplet_udft_drv.molecular_orbitals.print_orbitals(molecule, basis)

                                                                                                                          
                                             Spin Unrestricted Alpha Orbitals                                             
                                             --------------------------------                                             
                                                                                                                          
               Molecular Orbital No.   5:                                                                                 
               --------------------------                                                                                 
               Occupation: 1.000 Energy:   -0.56129 a.u.                                                                  
               (   1 O   1p+1:    -0.22) (   1 O   1p-1:    -0.42) (   1 O   2p-1:    -0.23)                              
               (

It is a common procedure to determine a DFT value estimating the expectation value of the  [total spin operator](https://kthpanor.github.io/echem/docs/elec_struct/spin.html#many-electron-systems) in the same manner as would be the case for a Hartree–Fock wave function.

```{note}
Since the [two-particle density](https://kthpanor.github.io/echem/docs/elec_struct/reduced_density.html#two-particle-density) is not directly available in DFT, the expectation value of a two-electron operator such as e.g. the total spin operator is also not available.
```

In [8]:
print(
    f"<S^2> RODFT = {triplet_rodft_drv.compute_s2(molecule, triplet_rodft_results) : 8.6f}"
)
print(
    f"<S^2> UDFT  = {triplet_udft_drv.compute_s2(molecule, triplet_udft_results) : 8.6f}"
)

<S^2> RODFT =  2.000000
<S^2> UDFT  =  2.006227


The ground state of oxygen is a triplet with spin quantum number $S = 1$ and

$$
\langle \hat{S}^2 \rangle = S(S+1)\hbar^2 = 2 \hbar^2
$$ 
 
This is exactly what we get with a restricted open-shell description, but not in the unrestricted case. In general, an unrestricted state is not a eigenfunction of $\hat{S}^2$ and one refers to this situation by saying that the state is spin contaminated. Implications of this will be discussed in more details in the section on low spin open-shell systems where the problem is more pronounced.

The relative simplicity of the unrestricted scheme together with its rather good performance makes it more widely used than the restricted open-shell form.

```{note}
The additional flexibility of unrestricted molecular orbitals does not lower the energy of *closed-shell* restricted states.
```

### Multi-reference methods

A multi-configurational method such as CASSCF can also be used and we note that a CASSCF wave function with only the high-spin open-shell MOs in the active space is equivalent to restricted open-shell Hartree–Fock (ROHF).

In [9]:
triplet_rohf_drv = vlx.ScfRestrictedOpenDriver()
triplet_rohf_results = triplet_rohf_drv.compute(molecule, basis)

# By default, OrbSpace includes all open-shells in the active space
space = mtp.OrbSpace(molecule, triplet_rohf_drv.mol_orbs)
triplet_mcscf_drv = mtp.McscfDriver()
triplet_mcscf_drv.compute(molecule, basis, space)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Open-Shell Hartree-Fock                              
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [10]:
print(f"       ROHF energy: {triplet_rohf_drv.get_scf_energy() : 14.8f}")
print(f"CASSCF(2,2) energy: {triplet_mcscf_drv.getEnergy() : 14.8f}")

       ROHF energy:  -149.60946112
CASSCF(2,2) energy:  -149.60946112


## Low spin states

### Single-reference methods: broken symmetry

To address low spin open-shell systems, we can study [singlet states of the oxygen molecule](https://en.wikipedia.org/wiki/Singlet_oxygen). The two lowest singlet states are $a^1\Delta_g$ and $b^1\Sigma^+_g$ at 22.5 and 37.5 kcal/mol above the ground triplet state, $X^3\Sigma_g^-$, respectively. 

Let us first compute the lowest closed-shell singlet state using DFT.

In [11]:
molecule.set_multiplicity(1)

singlet_rdft_drv = vlx.ScfRestrictedDriver()
singlet_rdft_drv.xcfun = "B3LYP"
singlet_rdft_results = singlet_rdft_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Kohn-Sham                                            
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

The $\pi$-orbitals in the closed-shell state are no longer doubly degenerate.

In [12]:
singlet_rdft_drv.molecular_orbitals.print_orbitals(molecule, basis)

                                                                                                                          
                                                 Spin Restricted Orbitals                                                 
                                                 ------------------------                                                 
                                                                                                                          
               Molecular Orbital No.   4:                                                                                 
               --------------------------                                                                                 
               Occupation: 2.000 Energy:   -0.79052 a.u.                                                                  
               (   1 O   2s  :    -0.36) (   1 O   3s  :    -0.46) (   2 O   2s  :     0.36)                              
               (

In [13]:
print("Closed-shell singlet relative to triplet ground state (in kcal/mol)")
print(
    f"    restricted triplet state: {(singlet_rdft_drv.get_scf_energy() - triplet_rodft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)
print(
    f"  unrestricted triplet state: {(singlet_rdft_drv.get_scf_energy() - triplet_udft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)

Closed-shell singlet relative to triplet ground state (in kcal/mol)
    restricted triplet state:    36.92114581
  unrestricted triplet state:    39.29447123


These energies match reasonably well with the higher $b^1\Sigma^+_g$ singlet state.

There are two different ways to reach the lower singlet using DFT. The first one is to compute the (negative) excitation energy using TD-DFT.

In [14]:
rsp_drv = vlx.TDAExciDriver()
rsp_drv.update_settings({"nstates": 1}, {"xcfun": "B3LYP"})
rsp_results = rsp_drv.compute(molecule, basis, singlet_rdft_results)

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 1                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [15]:
exc_energy = rsp_results["eigenvalues"][0]
print(f"Excitation energy (in kcal/mol): {exc_energy * au2kcal : 14.8}\n")

print("Open-shell singlet energy relative to triplet ground state (in kcal/mol)")
print(
    f"    restricted triplet state: {(singlet_rdft_drv.get_scf_energy() + exc_energy - triplet_rodft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)
print(
    f"  unrestricted triplet state: {(singlet_rdft_drv.get_scf_energy() + exc_energy - triplet_udft_drv.get_scf_energy()) * au2kcal : 14.8f}"
)

Excitation energy (in kcal/mol):     -11.732443

Open-shell singlet energy relative to triplet ground state (in kcal/mol)
    restricted triplet state:    25.18870298
  unrestricted triplet state:    27.56202840


These energies is in better agreement with the lowest singlet state. We have here reached the open-shell singlet state by means of a TD-DFT calculation based on a closed-shell reference state that is higher in energy.

Let us instead compute the lower singlet state in a direct manner using unrestricted DFT.

In [32]:
singlet_udft_drv = vlx.ScfUnrestrictedDriver()
singlet_udft_drv.xcfun = "B3LYP"
singlet_udft_result = singlet_udft_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Unrestricted Kohn-Sham                                          
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [33]:
print(f"RDFT singlet: {singlet_rdft_drv.get_scf_energy() : 14.8f}")
print(f"UDFT singlet: {singlet_udft_drv.get_scf_energy() : 14.8f}")

RDFT singlet:  -150.27131288
UDFT singlet:  -150.27131288


Since the closed-shell state represent a stable point, the UDFT solution will become identical. 

As there is no magnetic field, there is no impetus for the $\alpha$ and $\beta$ molecular orbitals to differ and the SCF optimization gets stuck in a symmetric solution. In the high-spin case, on the other hand, this symmetry was broken by the fact that we had an excess of $\alpha$-spin electrons.

However, we can break the symmetry by introducing different starting occupations for the $\alpha$ and $\beta$ spin-orbitals and remain in this region during the SCF optimization with the maximum-overlap method.

In [34]:
# A list of which orbitals are occupied in alpha and beta (starting from the triplet orbitals)
a_occ = [0, 1, 2, 3, 4, 5, 6, 7]
b_occ = [0, 1, 2, 3, 4, 5, 6, 8]

broken_udft_drv = vlx.ScfUnrestrictedDriver()
broken_udft_drv.xcfun = "B3LYP"

broken_udft_drv.maximum_overlap(
    molecule, basis, singlet_rdft_drv.mol_orbs, a_occ, b_occ
)
broken_udft_drv.conv_thresh = 1.0e-5
broken_udft_results = broken_udft_drv.compute(molecule, basis)

                                                                                                                          
* Info * Checkpoint written to file: veloxchem_scf_2022-12-09T18.05.51.scf.h5                                             
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Unrestricted Kohn-Sham                                          
                   Initial Guess Model             : Restart from Checkpoint                                              
                   Convergence Accelerator         : Direct Inversion of Iterative Subspace                               
                

The broken symmetry is noticeable in the orbital energies.

In [35]:
print(broken_udft_results["E_alpha"])
print(broken_udft_results["E_beta"])

[-19.27854022 -19.2783833   -1.28524019  -0.78801466  -0.54954777
  -0.52462322  -0.47281695  -0.28447192  -0.12169558   0.23668012
   0.79535564   0.81340929   0.85521247   0.89800219   0.9292196
   0.97198908   1.07236048   1.69840806   2.06485872   2.09877676
   2.34625043   2.34756336   2.65573861   2.6583274    2.81503837
   3.27997378   3.31937597   3.77952973]
[-19.27853995 -19.27838302  -1.28523984  -0.78801437  -0.54954744
  -0.52462312  -0.47281663  -0.28447146  -0.1216952    0.23668034
   0.7953557    0.81340949   0.85521266   0.89800212   0.9292197
   0.97198919   1.07236049   1.69840805   2.06485871   2.09877688
   2.34625061   2.34756351   2.65573881   2.65832759   2.81503855
   3.27997385   3.31937614   3.77952984]


We can look at the natural orbitals and see that the state indeed corresponds to two singly occupied $\pi$-orbitals.

In [36]:
broken_udft_drv.natural_orbitals().print_orbitals(molecule, basis)

                                                                                                                          
                                                 Spin Restricted Orbitals                                                 
                                                 ------------------------                                                 
                                                                                                                          
               Molecular Orbital No.   4:                                                                                 
               --------------------------                                                                                 
               Occupation: 2.000 Energy:   -1.01238 a.u.                                                                  
               (   1 O   2s  :     0.32) (   1 O   3s  :     0.38) (   2 O   2s  :     0.32)                              
               (

The energy of this singlet relative to the triplet ground state becomes:

In [37]:
print(
    f"Broken symmetry singlet-triplet gap:",
    (broken_udft_drv.get_scf_energy() - triplet_udft_drv.get_scf_energy()) * au2kcal,
)

Broken symmetry singlet-triplet gap: 10.307935117577651


With this trick, we have thus found a lower energy singlet, albeit by introducing a small degree of spin contamination.

In [38]:
print(
    f"<S^2> UDFT  = {broken_udft_drv.compute_s2(molecule, singlet_udft_results) : 8.6f}"
)

<S^2> UDFT  =  1.003282


We note that the energy gap (10.3 kcal/mol) is well below that of the literature reference (22.5 kcal/mol). The reason is that, while high spin open-shells are single-determinant in nature, open-shell singlets and low-spin triplets are not. 

A minimum of two determinant is needed for physically correct wave functions

\begin{align}
| \Psi_\mathrm{S} \rangle & = 
\frac{1}{\sqrt{2}} ( | \alpha \beta \rangle -| \beta \alpha \rangle ) \\
%
| \Psi_\mathrm{T} \rangle & = 
\frac{1}{\sqrt{2}} ( | \alpha \beta \rangle +| \beta \alpha \rangle )
\end{align}

Our single determinant reference state is seen to correspond to a 50/50 mix of the singlet and triplet wave functions as reflected in the spin contamination of the UDFT wavefunction.

True singlets and triplets have spin expectation values of $0$ and $2 \hbar^2$, respectively, whereas we obtained a value close to $\hbar^2$.

However, knowing this fact, we can actually correct the energy. Since we know the energy of the triplet and of the broken-symmetry solution, we can estimate the energy of the true singlet.

In [46]:
print(
    "Estimated singlet-triplet gap:",
    2
    * (broken_udft_drv.get_scf_energy() - triplet_udft_drv.get_scf_energy())
    * au2kcal,
)

Estimated singlet-triplet gap: 20.615870235155302


This is now very close to the experimental energy of 22.5 kcal/mol.

Our technique to obtain the "true" singlet energy is a special case of the more general "weighted-averaged broken symmetry" (WABS) method, which can in principle correct any broken symmetry solution with knowledge of the high-spin energy and the expectation value of the total spin operator.

### Multi-reference solution

In this case, the multi-reference solution is arguably simpler. You simply need to have an active space containing the right number of orbitals to describe the open-shells. For O$_2$, this is simply 2 electrons in 2 orbitals.

In [49]:
space = mtp.OrbSpace(molecule, singlet_rdft_drv.mol_orbs)
space.CAS(2, 2)
singlet_mcscf_drv = mtp.McscfDriver()
singlet_mcscf_drv.compute(molecule, basis, space)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 7
Number of active orbitals:              2
Number of virtual orbitals:             19

    This is a CASSCF wavefunction: CAS(2,2)

          CI expansion:
          -------------
Number of determinants:      3


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.555806448     0.0e+00      2.6e-01          1   0:00:00
        2     -149.561845052    -6.0e-03      4.4e-02          1   0:00:00
        3     -149.562109449    -2.6e-04      1.3e-02          1   0:00:00
      

In [55]:
print(
    f"Lowest singlet relative energy: {(singlet_mcscf_drv.getEnergy(0) - triplet_mcscf_drv.getEnergy()) * au2kcal : 14.8f}"
)

Lowest singlet relative energy:    29.69991700


The CASSCF automatically finds the open-shell to be the lowest state.

The CASSCF code in MultiPsi even allows to compute singlets and triplets simultaneously. We only need to deactivate the spin restriction that is applied by defaults to singlets and compute several states at the same time (here 4):

In [58]:
space = mtp.OrbSpace(molecule, singlet_rdft_drv.mol_orbs)
space.spinrestricted = False
space.CAS(2, 2)
all_mcscf_drv = mtp.McscfDriver()
all_mcscf_drv.compute(molecule, basis, space, nstates=4)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 7
Number of active orbitals:              2
Number of virtual orbitals:             19

    This is a CASSCF wavefunction: CAS(2,2)

          CI expansion:
          -------------
Number of determinants:      4


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.555736506     0.0e+00      2.6e-01          1   0:00:00
        2     -149.561864997    -6.1e-03      4.4e-02          1   0:00:00
        3     -149.562122529    -2.6e-04      1.2e-02          1   0:00:00
      

In [68]:
print(
    f"First singlet relative energy : {(all_mcscf_drv.getEnergy(1) - all_mcscf_drv.getEnergy(0)) * au2kcal : 14.8f}"
)
print(
    f"First singlet relative energy : {(all_mcscf_drv.getEnergy(2) - all_mcscf_drv.getEnergy(0)) * au2kcal : 14.8f}"
)
print(
    f"Second singlet relative energy: {(all_mcscf_drv.getEnergy(3) - all_mcscf_drv.getEnergy(0)) * au2kcal : 14.8f}"
)

First singlet relative energy :    29.55604879
First singlet relative energy :    29.55604879
Second singlet relative energy:    59.11209759


In a single calculation, the CASSCF finds the lowest state to be a triplet (see the spin multiplicity in the output), the following 2 states to be doubly degenerate singlets (the $a^1\Delta_g$ state is doubly degenerate) followed by a higher lying singlet. The energies are not a perfect match (CASSCF with such a small active space does not include correlation), but the order is correct.

Note that in this small molecule, a much better agreement with the experiment can be found by simply expanding the active space to include all 2p orbitals of the oxygen (resulting in six orbitals):

In [60]:
space.CAS(8, 6)
large_mcscf_drv = mtp.McscfDriver()
large_mcscf_drv.compute(molecule, basis, space, nstates=4)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 4
Number of active orbitals:              6
Number of virtual orbitals:             18

    This is a CASSCF wavefunction: CAS(8,6)

          CI expansion:
          -------------
Number of determinants:      225


                                                                                                                          
        MCSCF Iterations
        ----------------
                                                                                                                          
     Iter. | Average Energy | E. Change | Grad. Norm | CI Iter. |   Time
     ---------------------------------------------------------------------
        1     -149.640346807     0.0e+00      1.0e-01          4   0:00:00
        2     -149.665981557    -2.6e-02      7.1e-02          4   0:00:00
        3     -149.670480287    -4.5e-03      6.4e-02          4   0:00:00
    

In [69]:
print(
    f"First singlet relative energy : {(large_mcscf_drv.getEnergy(1) - large_mcscf_drv.getEnergy(0)) * au2kcal :14.8f}"
)
print(
    f"First singlet relative energy : {(large_mcscf_drv.getEnergy(2) - large_mcscf_drv.getEnergy(0)) * au2kcal : 14.8f}"
)
print(
    f"Second singlet relative energy: {(large_mcscf_drv.getEnergy(3) - large_mcscf_drv.getEnergy(0)) * au2kcal : 14.8f}"
)

First singlet relative energy :    21.00267674
First singlet relative energy :    21.00267674
Second singlet relative energy:    39.02173635


These results compare very favorably to the experimental results

- $a^1\Delta_g$: 22.5 kcal/mol
- $b^1\Sigma^+_g$: 37.5 kcal/mol

CASSCF also provides the wave functions, allowing us to clearly see the multi-determinant nature of the states.

In [62]:
for i, vec in enumerate(all_mcscf_drv.CIVecs):
    print("Wavefunction for state", i + 1)
    print(vec)

Wavefunction for state 1
Determinant     coef.    weight 
ab             -0.707    0.500 
ba              0.707    0.500 

Wavefunction for state 2
Determinant     coef.    weight 
20              0.707    0.500 
02             -0.707    0.500 

Wavefunction for state 3
Determinant     coef.    weight 
ab              0.707    0.500 
ba              0.707    0.500 

Wavefunction for state 4
Determinant     coef.    weight 
20              0.707    0.500 
02              0.707    0.500 



The determinant notation lists the occupation of the two orbitals in order, with "0" meaning "empty", "a" and "b" meaning respectively singly occupied with an $\alpha$ or $\beta$ electron, and "2" meaning doubly occupied. Clearly in this case, every single state in the calculation is an equal-weight superposition of 2 determinants.